# 🧪 01 - Data Merge Notebook
This notebook merges hourly and daily energy-related data for modeling energy prices in Germany.

In [38]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

data_dir = Path('../data/raw')
files = {
    'prices': data_dir / 'Germany.csv',
    'irradiance': data_dir / 'weather_data_filtered.csv',
    'production': data_dir / 'time_series_60min_singleindex_filtered.csv',
    'gas': data_dir / 'daily.csv',
    'oil': data_dir / 'RBRTEd.csv'
}

In [39]:
# Load all data
df_prices = pd.read_csv(files['prices'])
df_irradiance = pd.read_csv(files['irradiance'])
df_production = pd.read_csv(files['production'])
df_gas = pd.read_csv(files['gas'])
df_oil = pd.read_csv(files['oil'], names=['Date', 'Oil_Price'])

# Clean oil price data
df_oil['Date'] = pd.to_datetime(df_oil['Date'], errors='coerce')
df_oil = df_oil.dropna(subset=['Date'])
df_oil['Date'] = df_oil['Date'].dt.date
df_oil['Oil_Price'] = pd.to_numeric(df_oil['Oil_Price'], errors='coerce')
df_oil['Oil_Price'].fillna(method='ffill', inplace=True)

# Save cleaned oil file separately
df_oil.to_csv('../data/processed/cleaned_oil_prices.csv', index=False)

C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\4073196027.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_oil['Date'] = pd.to_datetime(df_oil['Date'], errors='coerce')
C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\4073196027.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_oil['Oil_Price'].fillna(method='ffill', inplace=True)
C:\Users\iTECH\AppData\Local\Temp\ipykernel

In [40]:
# Parse datetime fields and remove timezone info
df_prices['Datetime (UTC)'] = pd.to_datetime(df_prices['Datetime (UTC)']).dt.tz_localize(None)
df_irradiance['utc_timestamp'] = pd.to_datetime(df_irradiance['utc_timestamp']).dt.tz_localize(None)
df_production['utc_timestamp'] = pd.to_datetime(df_production['utc_timestamp']).dt.tz_localize(None)
df_gas['Date'] = pd.to_datetime(df_gas['Date'], errors='coerce')
df_gas['Date'] = df_gas['Date'].dt.date

In [41]:
# Merge hourly datasets
df_hourly = df_production.merge(df_prices, left_on='utc_timestamp', right_on='Datetime (UTC)', how='left')
df_hourly = df_hourly.merge(df_irradiance, on='utc_timestamp', how='left')

In [42]:
# Prepare date for merging daily data
df_hourly['date'] = df_hourly['utc_timestamp'].dt.date
df_gas.rename(columns={'Price': 'Gas_Price'}, inplace=True)

# Merge gas prices
df_hourly = df_hourly.merge(df_gas, left_on='date', right_on='Date', how='left')
df_hourly['Gas_Price'].fillna(method='ffill', inplace=True)

# Merge cleaned oil prices
df_hourly = df_hourly.merge(df_oil, left_on='date', right_on='Date', how='left')
df_hourly['Oil_Price'].fillna(method='ffill', inplace=True)
df_hourly['Oil_Price'].fillna(method='bfill', inplace=True)

C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\3068242119.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_hourly['Gas_Price'].fillna(method='ffill', inplace=True)
C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\3068242119.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_hourly['Gas_Price'].fillna(method='ffill', inplace=True)
C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\3068242119.py:11: FutureWarning: A value is try

In [43]:
# Drop redundant date columns safely
df_hourly.drop(columns=[col for col in ['Date', 'date'] if col in df_hourly.columns], inplace=True)

# Save final merged dataset
processed_path = Path('../data/processed/merged_energy_data.csv')
processed_path.parent.mkdir(parents=True, exist_ok=True)
df_hourly.to_csv(processed_path, index=False)
print(f'Saved merged dataset to: {processed_path}')

Saved merged dataset to: ..\data\processed\merged_energy_data.csv


In [44]:
df_hourly.head(3)

,utc_timestamp,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_load_forecast_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,...,Datetime (UTC),Datetime (Local),Price (EUR/MWhe),DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,Date_x,Gas_Price,Date_y,Oil_Price
0,2014-12-31 23:00:00,2015-01-01T00:00:00+0100,NaN,NaN,37248.0,0.0,0.0,27913.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,2014-12-31,3.14,NaN,55.38
1,2015-01-01 00:00:00,2015-01-01T01:00:00+0100,41151.0,39723.0,37248.0,0.0,0.0,27913.0,8852.0,0.3171,...,2015-01-01 00:00:00,2015-01-01 01:00:00,22.34,-0.981,0.0,0.0,NaN,3.14,NaN,55.38
2,2015-01-01 01:00:00,2015-01-01T02:00:00+0100,40135.0,38813.0,37248.0,0.0,0.0,27913.0,9054.0,0.3244,...,2015-01-01 01:00:00,2015-01-01 02:00:00,22.34,-1.035,0.0,0.0,NaN,3.14,NaN,55.38


In [45]:
df_hourly.drop(columns=['Datetime (UTC)', 'Datetime (Local)', 'Date_x', 'Date_y','Country','ISO3 Code'], inplace=True, errors='ignore')
df_hourly.head(3)

,utc_timestamp,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_load_forecast_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,...,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,Price (EUR/MWhe),DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,Gas_Price,Oil_Price
0,2014-12-31 23:00:00,2015-01-01T00:00:00+0100,NaN,NaN,37248.0,0.0,0.0,27913.0,NaN,NaN,...,NaN,27246.0,NaN,NaN,NaN,NaN,NaN,NaN,3.14,55.38
1,2015-01-01 00:00:00,2015-01-01T01:00:00+0100,41151.0,39723.0,37248.0,0.0,0.0,27913.0,8852.0,0.3171,...,0.7744,27246.0,8336.0,0.3059,22.34,-0.981,0.0,0.0,3.14,55.38
2,2015-01-01 01:00:00,2015-01-01T02:00:00+0100,40135.0,38813.0,37248.0,0.0,0.0,27913.0,9054.0,0.3244,...,0.7710,27246.0,8540.0,0.3134,22.34,-1.035,0.0,0.0,3.14,55.38


Checking for missing inside outside the range of study

In [47]:
# Define your date range
start_date = '2015-01-01'
end_date = '2019-12-30'

# Filter the DataFrame for that range
df_range = df_hourly[(df_hourly['utc_timestamp'] >= start_date) & (df_hourly['utc_timestamp'] <= end_date)]

# Check for any missing values
missing_summary = df_range.isnull().sum()

# Show only columns with missing data
print(missing_summary[missing_summary > 0])


DE_load_forecast_entsoe_transparency    24
DE_solar_profile                        96
DE_wind_generation_actual               74
DE_wind_profile                         74
DE_wind_offshore_generation_actual      74
DE_wind_offshore_profile                74
DE_wind_onshore_generation_actual       72
DE_wind_onshore_profile                 72
dtype: int64


In [48]:
df_range[df_range.isnull().any(axis=1)]

,utc_timestamp,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_load_forecast_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,...,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,Price (EUR/MWhe),DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,Gas_Price,Oil_Price
1393,2015-02-28 00:00:00,2015-02-28T01:00:00+0100,50517.0,50846.0,37435.0,0.0,NaN,28471.0,4391.0,0.1542,...,0.2418,27585.0,4177.0,0.1514,28.04,-0.789,0.0000,0.0000,2.79,61.89
1394,2015-02-28 01:00:00,2015-02-28T02:00:00+0100,49475.0,49596.0,37435.0,0.0,NaN,28471.0,4540.0,0.1595,...,0.3083,27585.0,4267.0,0.1547,27.07,-0.863,0.0000,0.0000,2.79,61.89
1395,2015-02-28 02:00:00,2015-02-28T03:00:00+0100,49074.0,46362.0,37435.0,0.0,NaN,28471.0,4299.0,0.1510,...,0.3212,27585.0,4014.0,0.1455,28.04,-0.931,0.0000,0.0000,2.79,61.89
1396,2015-02-28 03:00:00,2015-02-28T04:00:00+0100,48674.0,45530.0,37435.0,0.0,NaN,28471.0,4418.0,0.1552,...,0.4166,27585.0,4049.0,0.1468,27.30,-1.013,0.0000,0.0000,2.79,61.89
1397,2015-02-28 04:00:00,2015-02-28T05:00:00+0100,48654.0,46020.0,37435.0,0.0,NaN,28471.0,4643.0,0.1630,...,0.5984,27585.0,4113.0,0.1491,27.00,-1.115,0.0000,0.0000,2.79,61.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32706,2018-09-24 17:00:00,2018-09-24T19:00:00+0200,65814.0,NaN,45916.0,20.0,0.0004,47408.0,22746.0,0.4798,...,1.0400,43095.0,18260.0,0.4237,58.82,9.324,0.5497,3.6256,3.04,80.89
32707,2018-09-24 18:00:00,2018-09-24T20:00:00+0200,64202.0,NaN,45916.0,0.0,0.0000,47408.0,21663.0,0.4570,...,1.0248,43095.0,17243.0,0.4001,54.34,8.258,0.0000,0.0000,3.04,80.89
32708,2018-09-24 19:00:00,2018-09-24T21:00:00+0200,59629.0,NaN,45916.0,0.0,0.0000,47408.0,20693.0,0.4365,...,0.9854,43095.0,16443.0,0.3816,50.80,7.502,0.0000,0.0000,3.04,80.89
32709,2018-09-24 20:00:00,2018-09-24T22:00:00+0200,55391.0,NaN,45916.0,0.0,0.0000,47408.0,19175.0,0.4045,...,0.9451,43095.0,15099.0,0.3504,48.28,6.837,0.0000,0.0000,3.04,80.89


In [49]:
df_hourly = df_hourly[df_hourly['utc_timestamp'] <= '2019-12-30']

In [50]:
missing_solar = df_hourly[df_hourly['DE_solar_generation_actual'].isna()]
print(missing_solar['utc_timestamp'])

Series([], Name: utc_timestamp, dtype: datetime64[ns])


Will drop unneccessary or redundant columns even before EDA since it is obviously unuseful

In [51]:
print(df_hourly.columns)

Index(['utc_timestamp', 'cet_cest_timestamp',
       'DE_load_actual_entsoe_transparency',
       'DE_load_forecast_entsoe_transparency', 'DE_solar_capacity',
       'DE_solar_generation_actual', 'DE_solar_profile', 'DE_wind_capacity',
       'DE_wind_generation_actual', 'DE_wind_profile',
       'DE_wind_offshore_capacity', 'DE_wind_offshore_generation_actual',
       'DE_wind_offshore_profile', 'DE_wind_onshore_capacity',
       'DE_wind_onshore_generation_actual', 'DE_wind_onshore_profile',
       'Price (EUR/MWhe)', 'DE_temperature', 'DE_radiation_direct_horizontal',
       'DE_radiation_diffuse_horizontal', 'Gas_Price', 'Oil_Price'],
      dtype='object')


In [53]:
df_hourly.drop(columns=['DE_load_forecast_entsoe_transparency','DE_solar_profile','DE_wind_profile''DE_wind_offshore_capacity', 'DE_wind_offshore_generation_actual','DE_wind_offshore_profile', 'DE_wind_onshore_capacity','DE_wind_onshore_generation_actual', 'DE_wind_onshore_profile'], inplace=True, errors='ignore')
df_hourly.head(3)

,utc_timestamp,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,Price (EUR/MWhe),DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,Gas_Price,Oil_Price
0,2014-12-31 23:00:00,2015-01-01T00:00:00+0100,NaN,37248.0,0.0,27913.0,NaN,NaN,667.0,NaN,NaN,NaN,NaN,3.14,55.38
1,2015-01-01 00:00:00,2015-01-01T01:00:00+0100,41151.0,37248.0,0.0,27913.0,8852.0,0.3171,667.0,22.34,-0.981,0.0,0.0,3.14,55.38
2,2015-01-01 01:00:00,2015-01-01T02:00:00+0100,40135.0,37248.0,0.0,27913.0,9054.0,0.3244,667.0,22.34,-1.035,0.0,0.0,3.14,55.38


Checking missing data again

In [55]:
# Define your date range
start_date = '2015-01-01'
end_date = '2019-12-30'

# Filter the DataFrame for that range
df_range = df_hourly[(df_hourly['utc_timestamp'] >= start_date) & (df_hourly['utc_timestamp'] <= end_date)]

# Check for any missing values
missing_summary = df_range.isnull().sum()

# Show only columns with missing data
print(missing_summary[missing_summary > 0])

DE_wind_generation_actual    74
DE_wind_profile              74
dtype: int64


Interpolating to fill gaps

In [61]:
df_hourly['DE_wind_generation_actual'].interpolate(method='linear', inplace=True)
df_hourly['DE_wind_profile'].interpolate(method='linear', inplace=True)

C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\885302394.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_hourly['DE_wind_generation_actual'].interpolate(method='linear', inplace=True)
C:\Users\iTECH\AppData\Local\Temp\ipykernel_19132\885302394.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

In [62]:
# Define your date range
start_date = '2015-01-01'
end_date = '2019-12-30'

# Filter the DataFrame for that range
df_range = df_hourly[(df_hourly['utc_timestamp'] >= start_date) & (df_hourly['utc_timestamp'] <= end_date)]

# Check for any missing values
missing_summary = df_range.isnull().sum()

# Show only columns with missing data
print(missing_summary[missing_summary > 0])

Series([], dtype: int64)


In [63]:
processed_path = Path('../data/processed/merged_energy_data_final_step_1.csv')
processed_path.parent.mkdir(parents=True, exist_ok=True)
df_hourly.to_csv(processed_path, index=False)
print(f'Saved merged dataset to: {processed_path}')

Saved merged dataset to: ..\data\processed\merged_energy_data_final_step_1.csv


Now we have a data proper data set to model on </br>
Finished by Jad Akra on Friday 18th of April 2025